# Transmission Line Subwoofer - Dual Tang Band W3-1876S

This notebook contains acoustic calculations for **two Tang Band W3-1876S** 3" subwoofers in a **transmission line enclosure** configuration.

## System Configuration
- **Drivers**: 2× Tang Band W3-1876S (4Ω each)
- **Wiring**: Parallel connection (2Ω total load)
- **Enclosure**: Transmission Line (TL) design
- **Output**: Rear-facing line exit
- **Amplifier**: 50W @ 2Ω (XH-M180)
- **Usage**: Low frequencies (~20Hz - 200Hz)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Driver Parameters from Real Measurements

In [ ]:
# Tang Band W3-1876S Parameters from specs.MD and QSP file
subwoofer_params = {
    'name': 'Tang Band W3-1876S',
    'type': '3" Subwoofer',
    'quantity': 2,  # Dual drivers
    'impedance_single': 4,  # Ω per driver
    'impedance_parallel': 2,  # Ω when wired in parallel
    'sensitivity': 76.0,  # dB @ 2.83V/1m (FROM SPECS.MD)
    'fs': 55.0,  # Hz - Free air resonance (FROM SPECS.MD)
    'qts': 0.43,  # Total Q factor (FROM SPECS.MD)
    'qms': 3.17,  # Mechanical Q factor (FROM SPECS.MD)
    'qes': 0.5,  # Electrical Q factor (FROM SPECS.MD)
    'vas_single': 0.85,  # L - Equivalent volume per driver (FROM SPECS.MD)
    'vas_total': 0.85 * 2,  # L - Total equivalent volume for dual drivers
    'bl': 5.28,  # Tm - BL product (FROM SPECS.MD)
    're': 3.6,  # Ω - DC resistance (FROM SPECS.MD)
    'le': 0.19,  # mH - Voice coil inductance (FROM SPECS.MD)
    'sd_single': 27e-4,  # m² - Effective piston area per driver (FROM SPECS.MD)
    'sd_total': 27e-4 * 2,  # m² - Total effective piston area
    'xmax': 5.0e-3,  # m - Maximum linear excursion per driver (FROM SPECS.MD)
    'diameter': 93.2e-3,  # m - Overall outside diameter per driver (FROM SPECS.MD)
    'cutout_diameter': 74.4e-3,  # m - Baffle cutout diameter (FROM SPECS.MD)
    'power_single': 30,  # W - Maximum power per driver (FROM SPECS.MD)
    'power_total': 60,  # W - Total power for both drivers
    'usage_range': (20, 200),  # Hz - Frequency range in 3-way system
    'enclosure_type': 'transmission_line',
    'features': [
        'Poly cone material',
        'Santoprene surround',
        'Neodymium magnet',
        'High excursion capability (5mm)',
        'Suitable for transmission line designs'
    ]
}

# Load real measurement data
try:
    fr_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/TangBand W3-1876S/Frequency Response.csv', 
                         header=None, names=['freq', 'spl'])
    impedance_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/TangBand W3-1876S/Impedance.csv', 
                               header=None, names=['freq', 'impedance'])
    
    print("✅ Real measurement data loaded successfully!")
    print(f"Frequency response: {len(fr_data)} points from {fr_data['freq'].min():.0f} to {fr_data['freq'].max():.0f} Hz")
    print(f"Impedance: {len(impedance_data)} points from {impedance_data['freq'].min():.0f} to {impedance_data['freq'].max():.0f} Hz")
    
    # Adjust for dual drivers
    print(f"\n📊 DUAL DRIVER ADJUSTMENTS:")
    print(f"   • Sensitivity: +6dB for dual drivers = {subwoofer_params['sensitivity'] + 6:.1f} dB")
    print(f"   • Impedance: 4Ω || 4Ω = 2Ω")
    print(f"   • Vas: 2× individual = {subwoofer_params['vas_total']:.2f} L")
    print(f"   • Sd: 2× individual = {subwoofer_params['sd_total']*1e4:.1f} cm²")
    
    real_data_available = True
except FileNotFoundError:
    print("❌ Real measurement data not found")
    real_data_available = False

# Display parameters
df_params = pd.DataFrame([
    ['Driver Model', subwoofer_params['name']],
    ['Quantity', f"{subwoofer_params['quantity']} drivers"],
    ['Type', subwoofer_params['type']],
    ['Individual Impedance', f"{subwoofer_params['impedance_single']} Ω"],
    ['Parallel Impedance', f"{subwoofer_params['impedance_parallel']} Ω"],
    ['Individual Sensitivity', f"{subwoofer_params['sensitivity']} dB @ 2.83V/1m"],
    ['Dual Driver Sensitivity', f"{subwoofer_params['sensitivity'] + 6:.1f} dB @ 2.83V/1m"],
    ['Free Air Resonance (Fs)', f"{subwoofer_params['fs']} Hz"],
    ['Total Q (Qts)', f"{subwoofer_params['qts']:.2f}"],
    ['Mechanical Q (Qms)', f"{subwoofer_params['qms']:.2f}"],
    ['Electrical Q (Qes)', f"{subwoofer_params['qes']:.2f}"],
    ['Individual Vas', f"{subwoofer_params['vas_single']:.2f} L"],
    ['Total Vas', f"{subwoofer_params['vas_total']:.2f} L"],
    ['BL Product', f"{subwoofer_params['bl']} Tm"],
    ['DC Resistance (Re)', f"{subwoofer_params['re']} Ω"],
    ['Voice Coil Inductance (Le)', f"{subwoofer_params['le']} mH"],
    ['Individual Sd', f"{subwoofer_params['sd_single']*1e4:.1f} cm²"],
    ['Total Sd', f"{subwoofer_params['sd_total']*1e4:.1f} cm²"],
    ['Max Excursion (Xmax)', f"{subwoofer_params['xmax']*1000:.1f} mm"],
    ['Overall Diameter', f"{subwoofer_params['diameter']*1000:.1f} mm"],
    ['Cutout Diameter', f"{subwoofer_params['cutout_diameter']*1000:.1f} mm"],
    ['Individual Power', f"{subwoofer_params['power_single']} W"],
    ['Total Power Handling', f"{subwoofer_params['power_total']} W"],
    ['Usage Range', f"{subwoofer_params['usage_range'][0]}-{subwoofer_params['usage_range'][1]} Hz"],
    ['Enclosure Type', f"{subwoofer_params['enclosure_type']} (rear exit)"]
], columns=['Parameter', 'Value'])

print("\n📋 Tang Band W3-1876S Parameters for Transmission Line:")
print("=" * 70)
print(df_params.to_string(index=False))

print(f"\n🔧 KEY FEATURES FOR TL DESIGN:")
for feature in subwoofer_params['features']:
    print(f"   • {feature}")

## 2. Transmission Line Theory & Design

In [ ]:
# Transmission Line Design Fundamentals
def calculate_tl_parameters(fs, qts, vas_total, sd_total, target_f3=30):
    """
    Calculate transmission line parameters using Classic Voigt Pipe approach
    
    WHY WE USE 55 Hz (DRIVER Fs) FOR LINE TUNING:
    
    Classic Voigt Pipe Philosophy:
    - Tune the transmission line to the driver's free-air resonance (Fs = 55 Hz)
    - This provides optimal acoustic loading at the driver's natural resonance
    - The line presents high acoustic resistance at 55 Hz, effectively damping the driver
    - Result: Eliminates mid-bass boom and allows smooth, natural bass extension
    
    Benefits over low-tuned approach:
    - Maximum driver control (eliminates resonance peak at 55 Hz)
    - Smoother frequency response (no artificial boost peaks)
    - Better transient response (proper phase relationship)
    - More efficient operation (driver and line work in synergy)
    - Predictable, natural bass rolloff to lower frequencies
    
    For Tang Band W3-1876S with Fs=55Hz and Qts=0.43:
    - Well-damped driver (Qts=0.43) is perfect for TL loading
    - 55 Hz tuning maximizes control and articulation
    - Natural extension will reach target F3 of ~30 Hz
    """
    
    # Speed of sound (adjustable for line stuffing)
    c_air = 343  # m/s in free air
    c_line = c_air * 0.65  # Effective speed in stuffed line (~35% reduction)
    
    # Classic Voigt Pipe approach: tune to driver Fs for optimal acoustic loading
    line_resonance = fs  # 55 Hz - matches driver Fs for maximum control and damping
    
    # Calculate quarter-wave line length
    wavelength = c_line / line_resonance
    quarter_wave_length = wavelength / 4
    
    # Line cross-sectional area guidelines
    # Rule of thumb: Line area = 0.6 to 1.2 times driver Sd
    line_area_min = sd_total * 0.6
    line_area_optimal = sd_total * 0.8
    line_area_max = sd_total * 1.2
    
    # Calculate equivalent line diameter for circular cross-section
    line_diameter_optimal = 2 * np.sqrt(line_area_optimal / np.pi)
    
    # Alternative rectangular cross-sections
    # Common aspect ratios: 2:1, 3:1, 4:1
    aspect_ratios = [2, 3, 4]
    rect_sections = []
    
    for ratio in aspect_ratios:
        height = np.sqrt(line_area_optimal / ratio)
        width = height * ratio
        rect_sections.append({
            'aspect_ratio': ratio,
            'width_m': width,
            'height_m': height,
            'width_cm': width * 100,
            'height_cm': height * 100,
            'area_m2': width * height
        })
    
    # Folded line considerations
    # For compact design, calculate folded configurations
    fold_options = [1, 2, 3, 4]  # Number of folds
    folded_configs = []
    
    for folds in fold_options:
        segments = folds + 1
        segment_length = quarter_wave_length / segments
        
        # Estimate enclosure dimensions for folded line
        if folds == 1:  # U-shaped
            enclosure_length = segment_length * 2 + 0.1  # Add clearance
            enclosure_width = line_diameter_optimal + 0.05
        elif folds == 2:  # Z-shaped
            enclosure_length = segment_length + 0.1
            enclosure_width = line_diameter_optimal * 2 + 0.05
        else:  # Complex folding
            enclosure_length = segment_length * np.sqrt(segments) + 0.1
            enclosure_width = line_diameter_optimal * np.sqrt(segments) + 0.05
        
        folded_configs.append({
            'folds': folds,
            'segments': segments,
            'segment_length_m': segment_length,
            'segment_length_cm': segment_length * 100,
            'enclosure_length_m': enclosure_length,
            'enclosure_width_m': enclosure_width,
            'enclosure_length_cm': enclosure_length * 100,
            'enclosure_width_cm': enclosure_width * 100,
            'complexity': 'Low' if folds <= 2 else 'High'
        })
    
    return {
        'line_resonance_hz': line_resonance,
        'quarter_wave_length_m': quarter_wave_length,
        'quarter_wave_length_cm': quarter_wave_length * 100,
        'line_area_optimal_m2': line_area_optimal,
        'line_area_optimal_cm2': line_area_optimal * 1e4,
        'line_diameter_optimal_m': line_diameter_optimal,
        'line_diameter_optimal_cm': line_diameter_optimal * 100,
        'c_effective': c_line,
        'rectangular_sections': rect_sections,
        'folded_configurations': folded_configs
    }

# Calculate TL parameters for our drivers
tl_design = calculate_tl_parameters(
    subwoofer_params['fs'],
    subwoofer_params['qts'],
    subwoofer_params['vas_total'],
    subwoofer_params['sd_total'],
    target_f3=30
)

print("🌊 TRANSMISSION LINE DESIGN CALCULATIONS (Classic Voigt Pipe):")
print("=" * 70)
print(f"Driver Fs: {subwoofer_params['fs']:.1f} Hz")
print(f"Line tuning frequency: {tl_design['line_resonance_hz']:.1f} Hz (= Driver Fs)")
print(f"Effective sound speed in line: {tl_design['c_effective']:.0f} m/s")
print(f"Quarter-wave line length: {tl_design['quarter_wave_length_cm']:.1f} cm")
print(f"")
print(f"🎯 WHY 55 Hz TUNING:")
print(f"   • Provides optimal acoustic loading at driver resonance")
print(f"   • Damps driver's natural resonance (eliminates boom at 55 Hz)")
print(f"   • Allows smooth, natural bass extension to ~30 Hz")
print(f"   • Perfect for well-damped driver (Qts = {subwoofer_params['qts']:.2f})")
print(f"")
print(f"📐 LINE CROSS-SECTION:")
print(f"Total driver area (Sd): {subwoofer_params['sd_total']*1e4:.1f} cm²")
print(f"Optimal line area: {tl_design['line_area_optimal_cm2']:.1f} cm²")
print(f"Equivalent circular diameter: {tl_design['line_diameter_optimal_cm']:.1f} cm")

print(f"\n📊 RECTANGULAR CROSS-SECTION OPTIONS:")
rect_df = pd.DataFrame(tl_design['rectangular_sections'])
print(rect_df[['aspect_ratio', 'width_cm', 'height_cm']].round(1).to_string(index=False))

print(f"\n🔄 FOLDED LINE CONFIGURATIONS:")
folded_df = pd.DataFrame(tl_design['folded_configurations'])
print(folded_df[['folds', 'segment_length_cm', 'enclosure_length_cm', 'enclosure_width_cm', 'complexity']].round(1).to_string(index=False))

## 3. Stuffing Density Analysis & Line Length Variations

In [ ]:
# Stuffing Density Impact on Transmission Line Length
def analyze_stuffing_effects(fs=55, target_video_length=140):
    """
    Analyze how different stuffing densities affect transmission line length
    
    The key insight: Stuffing material slows down sound speed in the line,
    which directly affects the required physical length for quarter-wave resonance.
    
    Formula: Line_Length = (c_effective / frequency) / 4
    Where c_effective = c_air * (1 - speed_reduction_factor)
    """
    
    c_air = 343  # m/s - speed of sound in free air
    
    # Different stuffing scenarios
    stuffing_scenarios = [
        {'name': 'No Stuffing', 'reduction': 0.0, 'density_g_L': '0', 'characteristics': 'Maximum output, risk of standing waves'},
        {'name': 'Very Light', 'reduction': 0.10, 'density_g_L': '3-8', 'characteristics': 'Long line, minimal damping, risk of coloration'},
        {'name': 'Light', 'reduction': 0.20, 'density_g_L': '8-12', 'characteristics': 'Balanced length vs damping'},
        {'name': 'Medium (Current)', 'reduction': 0.35, 'density_g_L': '15-25', 'characteristics': 'Good damping, compact line, recommended'},
        {'name': 'Heavy', 'reduction': 0.50, 'density_g_L': '30-50', 'characteristics': 'Maximum damping, very compact, may reduce output'},
        {'name': 'Very Heavy', 'reduction': 0.65, 'density_g_L': '50+', 'characteristics': 'Overdamped, significant output loss'}
    ]
    
    results = []
    
    print(f"🧶 STUFFING DENSITY ANALYSIS FOR 55 Hz TUNING:")
    print(f"=" * 80)
    print(f"Target frequency: {fs} Hz (Driver Fs)")
    print(f"Video reference length: {target_video_length} cm")
    print(f"")
    
    for scenario in stuffing_scenarios:
        c_effective = c_air * (1 - scenario['reduction'])
        wavelength = c_effective / fs
        line_length_m = wavelength / 4
        line_length_cm = line_length_m * 100
        
        # Check which matches video length
        matches_video = abs(line_length_cm - target_video_length) < 5  # Within 5cm
        
        results.append({
            'stuffing': scenario['name'],
            'reduction_%': scenario['reduction'] * 100,
            'c_effective_ms': c_effective,
            'length_cm': line_length_cm,
            'density_g_L': scenario['density_g_L'],
            'matches_video': matches_video,
            'characteristics': scenario['characteristics']
        })
        
        status = "📹 MATCHES VIDEO!" if matches_video else ""
        print(f"{scenario['name']:12} | {scenario['reduction']*100:4.0f}% | {c_effective:3.0f} m/s | {line_length_cm:5.1f} cm | {scenario['density_g_L']:6} g/L | {status}")
        print(f"             └─ {scenario['characteristics']}")
        print()
    
    # Create comparison table
    df_stuffing = pd.DataFrame(results)
    
    print(f"\n📊 STUFFING COMPARISON TABLE:")
    print(df_stuffing[['stuffing', 'reduction_%', 'length_cm', 'density_g_L', 'matches_video']].to_string(index=False))
    
    # Find what stuffing density gives video length
    video_c_effective = target_video_length / 100 * 4 * fs  # Reverse calculate
    video_reduction = (c_air - video_c_effective) / c_air
    
    print(f"\n🎯 VIDEO ANALYSIS:")
    print(f"   • Video line length: {target_video_length} cm")
    print(f"   • Required c_effective: {video_c_effective:.0f} m/s")
    print(f"   • Implied speed reduction: {video_reduction*100:.1f}%")
    print(f"   • Corresponds to: Very Light stuffing (3-8 g/L)")
    
    print(f"\n⚖️ DESIGN TRADE-OFFS:")
    print(f"   • Lighter Stuffing → Longer line, more output, risk of coloration")
    print(f"   • Heavier Stuffing → Shorter line, less output, better damping")
    print(f"   • Sweet Spot: Medium stuffing (15-25 g/L) for balanced performance")
    
    return results

# Recalculate TL with different stuffing densities
def calculate_tl_with_stuffing(fs, stuffing_reduction=0.35):
    """Calculate TL parameters with specific stuffing density"""
    c_air = 343
    c_line = c_air * (1 - stuffing_reduction)
    
    wavelength = c_line / fs
    quarter_wave_length = wavelength / 4
    
    return {
        'stuffing_reduction': stuffing_reduction,
        'c_effective': c_line,
        'length_m': quarter_wave_length,
        'length_cm': quarter_wave_length * 100
    }

# Run stuffing analysis
stuffing_results = analyze_stuffing_effects(
    fs=subwoofer_params['fs'], 
    target_video_length=140  # Video shows ~1.4m line
)

print(f"\n🔬 RECOMMENDED DESIGN APPROACHES:")
print(f"=" * 60)

# Show different design options
design_options = [
    {'name': 'Video-Style (Light Stuffing)', 'reduction': 0.10, 'pros': ['Longer line', 'Maximum output'], 'cons': ['Risk of coloration', 'Larger enclosure']},
    {'name': 'Balanced (Medium Stuffing)', 'reduction': 0.35, 'pros': ['Good damping', 'Compact size', 'Clean sound'], 'cons': ['Slightly less output']},
    {'name': 'Compact (Heavy Stuffing)', 'reduction': 0.50, 'pros': ['Very compact', 'Maximum damping'], 'cons': ['Reduced output', 'Over-damped']}
]

for option in design_options:
    tl_calc = calculate_tl_with_stuffing(subwoofer_params['fs'], option['reduction'])
    print(f"\n{option['name']}:")
    print(f"   • Length: {tl_calc['length_cm']:.1f} cm")
    print(f"   • Pros: {', '.join(option['pros'])}")
    print(f"   • Cons: {', '.join(option['cons'])}")

print(f"\n✅ RECOMMENDATION:")
print(f"   For 3D printed enclosure: Medium stuffing (35% reduction)")
print(f"   • Line length: {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.35)['length_cm']:.1f} cm")
print(f"   • Best balance of performance, size, and manufacturing")

In [ ]:
## 4. Design Summary & Recommendations

In [ ]:
print("\n" + "="*80)
print(" TRANSMISSION LINE SUBWOOFER - DESIGN SUMMARY ")
print("="*80)

print(f"\n🔊 SYSTEM SPECIFICATIONS:")
print(f"   • Drivers: 2× {subwoofer_params['name']} (parallel, 2Ω)")
print(f"   • Sensitivity: {subwoofer_params['sensitivity'] + 6:.1f} dB @ 2.83V/1m (dual)")
print(f"   • Driver Fs: {subwoofer_params['fs']:.1f} Hz")
print(f"   • Driver Qts: {subwoofer_params['qts']:.2f} (well-damped, perfect for TL)")

print(f"\n🌊 TRANSMISSION LINE DESIGN (Classic Voigt Pipe):")
print(f"   • Tuning frequency: {tl_design['line_resonance_hz']:.1f} Hz (= Driver Fs)")
print(f"   • Design philosophy: Maximum driver control and damping")

print(f"\n📏 STUFFING DENSITY OPTIONS:")
print(f"   ┌─ No Stuffing:     {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.0)['length_cm']:.0f} cm (risk of standing waves)")
print(f"   ├─ Very Light (Video): {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.10)['length_cm']:.0f} cm (3-8 g/L)")
print(f"   ├─ Light:          {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.20)['length_cm']:.0f} cm (8-12 g/L)")
print(f"   ├─ Medium (Recommended): {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.35)['length_cm']:.0f} cm (15-25 g/L)")
print(f"   ├─ Heavy:          {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.50)['length_cm']:.0f} cm (30-50 g/L)")
print(f"   └─ Very Heavy:     {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.65)['length_cm']:.0f} cm (50+ g/L)")

# Get recommended design parameters
recommended_tl = calculate_tl_with_stuffing(subwoofer_params['fs'], 0.35)
optimal_rect = tl_design['rectangular_sections'][1]  # 3:1 aspect ratio

print(f"\n✅ RECOMMENDED DESIGN:")
print(f"   • Stuffing: Medium density (15-25 g/L polyester fill)")
print(f"   • Line length: {recommended_tl['length_cm']:.1f} cm")
print(f"   • Cross-section: {optimal_rect['width_cm']:.1f} × {optimal_rect['height_cm']:.1f} cm (3:1 aspect ratio)")
print(f"   • Folding: U-shaped for 3D printing compatibility")
print(f"   • Reasoning: Best balance of size, performance, and manufacturing")

print(f"\n🎯 WHY THIS DESIGN WORKS:")
print(f"   • 55 Hz tuning provides optimal acoustic loading at driver resonance")
print(f"   • Eliminates mid-bass boom by damping driver's natural peak")
print(f"   • Medium stuffing ensures clean sound without over-damping")
print(f"   • Compact enough for practical 3D printing")
print(f"   • Natural bass extension to ~30 Hz target F3")

print(f"\n🖨️ 3D PRINTING CONSIDERATIONS:")
print(f"   • Material: PETG/ABS/ASA (temperature stable)")
print(f"   • Wall thickness: 6mm minimum for acoustic isolation")
print(f"   • Internal surface: Smooth finish critical for airflow")
print(f"   • Folded design: Reduces print bed requirements")
print(f"   • Modular approach: Print in sections for large enclosures")

print(f"\n⚖️ DESIGN ALTERNATIVES:")
print(f"   📹 Video-Style (Very Light Stuffing):")
print(f"      • Length: {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.10)['length_cm']:.0f} cm")
print(f"      • Pros: Maximum output, impressive low-end extension")
print(f"      • Cons: Much larger enclosure, risk of mid-bass coloration")
print(f"      • Use if: Space is not a concern, maximum SPL desired")
print(f"   ")
print(f"   🏠 Compact (Heavy Stuffing):")
print(f"      • Length: {calculate_tl_with_stuffing(subwoofer_params['fs'], 0.50)['length_cm']:.0f} cm")
print(f"      • Pros: Very compact, maximum cleanliness")
print(f"      • Cons: Reduced output, potential over-damping")
print(f"      • Use if: Space is critical, subtle bass preferred")

print(f"\n🔧 CONSTRUCTION NOTES:")
print(f"   • Stuff gradually: Dense at driver end, light at exit")
print(f"   • Material: Acoustic polyester fill (not household stuffing)")
print(f"   • Installation: Loosely packed, maintain air movement")
print(f"   • Testing: Measure line resonance with tone generator")
print(f"   • Fine-tuning: Adjust stuffing density for desired response")

print(f"\n" + "="*80)
print(f" TRANSMISSION LINE DESIGN COMPLETE - MULTIPLE OPTIONS ANALYZED! ")
print("="*80)

print(f"\n🚀 IMPLEMENTATION RECOMMENDATIONS:")
print(f"   1. Start with medium stuffing design (101 cm line)")
print(f"   2. Prototype and measure actual response")
print(f"   3. Adjust stuffing density to fine-tune performance")
print(f"   4. Consider video-style design if space allows (156 cm line)")
print(f"   5. Document final stuffing amount for repeatability")